### Initialization

In [8]:
import theano
import lasagne
import time
import os
import random

import numpy as np
import pickle as cp
import theano.tensor as T

SENSOR_CHANNELS = 42
NUM_CLASSES_GCSm = 6
NUM_CLASSES_GCSe = 4
NUM_CLASSES_GCSv = 5
NUM_CLASSES_GCST = 13
WINDOW_LENGTH = 721
FINAL_SEQUENCE_LENGTH = 705
BATCH_SIZE = 32
NUM_FILTERS = 64
FILTER_SIZE = 5
NUM_UNITS_LSTM = 128
EPOCHS = 100

input_var = T.tensor4('input')
target_var = T.ivector('targets')

### Load Sensor Data and GCS Labels

In [ ]:
baseDir = os.path.expanduser('~/data/all_motion_feature_data/train_test_sets/')

trainArrayGCSe = np.load(os.path.join(baseDir,"trainArrayGCSe.npy"))
trainArrayGCSm = np.load(os.path.join(baseDir,"trainArrayGCSm.npy"))
trainArrayGCSv = np.load(os.path.join(baseDir,"trainArrayGCSv.npy"))
trainArrayGCST = np.load(os.path.join(baseDir,"trainArrayGCST.npy"))

testArrayGCSe = np.load(os.path.join(baseDir,"testArrayGCSe.npy"))
testArrayGCSm = np.load(os.path.join(baseDir,"testArrayGCSm.npy"))
testArrayGCSv = np.load(os.path.join(baseDir,"testArrayGCSv.npy"))
testArrayGCST = np.load(os.path.join(baseDir,"testArrayGCST.npy"))

trainLabelsGCSe = np.load(os.path.join(baseDir,"trainLabelsGCSe.npy"))
trainLabelsGCSm = np.load(os.path.join(baseDir,"trainLabelsGCSm.npy"))
trainLabelsGCSv = np.load(os.path.join(baseDir,"trainLabelsGCSv.npy"))
trainLabelsGCST = np.load(os.path.join(baseDir,"trainLabelsGCST.npy"))

testLabelsGCSe = np.load(os.path.join(baseDir,"testLabelsGCSe.npy"))
testLabelsGCSm = np.load(os.path.join(baseDir,"testLabelsGCSm.npy"))
testLabelsGCSv = np.load(os.path.join(baseDir,"testLabelsGCSv.npy"))
testLabelsGCST = np.load(os.path.join(baseDir,"testLabelsGCST.npy"))

### Define the Lasagne Networks

In [ ]:
# Motor GCS Score Network
GCSm_net = {}
GCSm_net['input'] = lasagne.layers.InputLayer((BATCH_SIZE, 1, WINDOW_LENGTH, SENSOR_CHANNELS),input_var=input_var)
GCSm_net['conv1/5x1'] = lasagne.layers.Conv2DLayer(GCSm_net['input'], NUM_FILTERS, (FILTER_SIZE, 1))
GCSm_net['conv2/5x1'] = lasagne.layers.Conv2DLayer(GCSm_net['conv1/5x1'], NUM_FILTERS, (FILTER_SIZE, 1))
GCSm_net['conv3/5x1'] = lasagne.layers.Conv2DLayer(GCSm_net['conv2/5x1'], NUM_FILTERS, (FILTER_SIZE, 1))
GCSm_net['conv4/5x1'] = lasagne.layers.Conv2DLayer(GCSm_net['conv3/5x1'], NUM_FILTERS, (FILTER_SIZE, 1))
GCSm_net['shuff'] = lasagne.layers.DimshuffleLayer(GCSm_net['conv4/5x1'], (0, 2, 1, 3))
GCSm_net['lstm1'] = lasagne.layers.LSTMLayer(GCSm_net['shuff'], NUM_UNITS_LSTM)
GCSm_net['lstm2'] = lasagne.layers.LSTMLayer(GCSm_net['lstm1'], NUM_UNITS_LSTM)
GCSm_net['shp1'] = lasagne.layers.ReshapeLayer(GCSm_net['lstm2'], (-1, NUM_UNITS_LSTM))
GCSm_net['prob'] = lasagne.layers.DenseLayer(GCSm_net['shp1'],NUM_CLASSES_GCSm, nonlinearity=lasagne.nonlinearities.softmax)
GCSm_net['shp2'] = lasagne.layers.ReshapeLayer(GCSm_net['prob'], (BATCH_SIZE, FINAL_SEQUENCE_LENGTH, NUM_CLASSES_GCSm))
GCSm_net['output'] = lasagne.layers.SliceLayer(GCSm_net['shp2'], -1, 1)

In [ ]:
# Eye GCS Score Network
GCSe_net = {}
GCSe_net['input'] = lasagne.layers.InputLayer((BATCH_SIZE, 1, WINDOW_LENGTH, SENSOR_CHANNELS),input_var=input_var)
GCSe_net['conv1/5x1'] = lasagne.layers.Conv2DLayer(GCSe_net['input'], NUM_FILTERS, (FILTER_SIZE, 1))
GCSe_net['conv2/5x1'] = lasagne.layers.Conv2DLayer(GCSe_net['conv1/5x1'], NUM_FILTERS, (FILTER_SIZE, 1))
GCSe_net['conv3/5x1'] = lasagne.layers.Conv2DLayer(GCSe_net['conv2/5x1'], NUM_FILTERS, (FILTER_SIZE, 1))
GCSe_net['conv4/5x1'] = lasagne.layers.Conv2DLayer(GCSe_net['conv3/5x1'], NUM_FILTERS, (FILTER_SIZE, 1))
GCSe_net['shuff'] = lasagne.layers.DimshuffleLayer(GCSe_net['conv4/5x1'], (0, 2, 1, 3))
GCSe_net['lstm1'] = lasagne.layers.LSTMLayer(GCSe_net['shuff'], NUM_UNITS_LSTM)
GCSe_net['lstm2'] = lasagne.layers.LSTMLayer(GCSe_net['lstm1'], NUM_UNITS_LSTM)
GCSe_net['shp1'] = lasagne.layers.ReshapeLayer(GCSe_net['lstm2'], (-1, NUM_UNITS_LSTM))
GCSe_net['prob'] = lasagne.layers.DenseLayer(GCSe_net['shp1'],NUM_CLASSES_GCSe, nonlinearity=lasagne.nonlinearities.softmax)
GCSe_net['shp2'] = lasagne.layers.ReshapeLayer(GCSe_net['prob'], (BATCH_SIZE, FINAL_SEQUENCE_LENGTH, NUM_CLASSES_GCSe))
GCSe_net['output'] = lasagne.layers.SliceLayer(GCSe_net['shp2'], -1, 1)

In [ ]:
# Verbal GCS Score Network
GCSv_net = {}
GCSv_net['input'] = lasagne.layers.InputLayer((BATCH_SIZE, 1, WINDOW_LENGTH, SENSOR_CHANNELS),input_var=input_var)
GCSv_net['conv1/5x1'] = lasagne.layers.Conv2DLayer(GCSv_net['input'], NUM_FILTERS, (FILTER_SIZE, 1))
GCSv_net['conv2/5x1'] = lasagne.layers.Conv2DLayer(GCSv_net['conv1/5x1'], NUM_FILTERS, (FILTER_SIZE, 1))
GCSv_net['conv3/5x1'] = lasagne.layers.Conv2DLayer(GCSv_net['conv2/5x1'], NUM_FILTERS, (FILTER_SIZE, 1))
GCSv_net['conv4/5x1'] = lasagne.layers.Conv2DLayer(GCSv_net['conv3/5x1'], NUM_FILTERS, (FILTER_SIZE, 1))
GCSv_net['shuff'] = lasagne.layers.DimshuffleLayer(GCSv_net['conv4/5x1'], (0, 2, 1, 3))
GCSv_net['lstm1'] = lasagne.layers.LSTMLayer(GCSv_net['shuff'], NUM_UNITS_LSTM)
GCSv_net['lstm2'] = lasagne.layers.LSTMLayer(GCSv_net['lstm1'], NUM_UNITS_LSTM)
GCSv_net['shp1'] = lasagne.layers.ReshapeLayer(GCSv_net['lstm2'], (-1, NUM_UNITS_LSTM))
GCSv_net['prob'] = lasagne.layers.DenseLayer(GCSv_net['shp1'],NUM_CLASSES_GCSv, nonlinearity=lasagne.nonlinearities.softmax)
GCSv_net['shp2'] = lasagne.layers.ReshapeLayer(GCSv_net['prob'], (BATCH_SIZE, FINAL_SEQUENCE_LENGTH, NUM_CLASSES_GCSv))
GCSv_net['output'] = lasagne.layers.SliceLayer(GCSv_net['shp2'], -1, 1)

In [ ]:
# Total GCS Score Network
GCST_net = {}
GCST_net['input'] = lasagne.layers.InputLayer((BATCH_SIZE, 1, WINDOW_LENGTH, SENSOR_CHANNELS),input_var=input_var)
GCST_net['conv1/5x1'] = lasagne.layers.Conv2DLayer(GCST_net['input'], NUM_FILTERS, (FILTER_SIZE, 1))
GCST_net['conv2/5x1'] = lasagne.layers.Conv2DLayer(GCST_net['conv1/5x1'], NUM_FILTERS, (FILTER_SIZE, 1))
GCST_net['conv3/5x1'] = lasagne.layers.Conv2DLayer(GCST_net['conv2/5x1'], NUM_FILTERS, (FILTER_SIZE, 1))
GCST_net['conv4/5x1'] = lasagne.layers.Conv2DLayer(GCST_net['conv3/5x1'], NUM_FILTERS, (FILTER_SIZE, 1))
GCST_net['shuff'] = lasagne.layers.DimshuffleLayer(GCST_net['conv4/5x1'], (0, 2, 1, 3))
GCST_net['lstm1'] = lasagne.layers.LSTMLayer(GCST_net['shuff'], NUM_UNITS_LSTM)
GCST_net['lstm2'] = lasagne.layers.LSTMLayer(GCST_net['lstm1'], NUM_UNITS_LSTM)
GCST_net['shp1'] = lasagne.layers.ReshapeLayer(GCST_net['lstm2'], (-1, NUM_UNITS_LSTM))
GCST_net['prob'] = lasagne.layers.DenseLayer(GCST_net['shp1'],NUM_CLASSES_GCST, nonlinearity=lasagne.nonlinearities.softmax)
GCST_net['shp2'] = lasagne.layers.ReshapeLayer(GCST_net['prob'], (BATCH_SIZE, FINAL_SEQUENCE_LENGTH, NUM_CLASSES_GCST))
GCST_net['output'] = lasagne.layers.SliceLayer(GCST_net['shp2'], -1, 1)

### Defining training parameters

In [ ]:
# Defining hyperparameters
lr = 1e-2
weight_decay = 1e-5

prediction = lasagne.layers.get_output(GCSm_net['output'])
loss = lasagne.objectives.categorical_crossentropy(prediction, target_var)
loss = loss.mean()

weightsl2 = lasagne.regularization.regularize_network_params(GCSm_net['output'], lasagne.regularization.l2)
loss += weight_decay * weightsl2

#Get the update rule for Stochastic Gradient Descent with ADAM
params = lasagne.layers.get_all_params(GCSm_net['output'], trainable=True)
updates = lasagne.updates.adam(loss, params, learning_rate=lr)

# Defining training and diagnostics functions:
train_fn = theano.function([input_var, target_var], loss, updates=updates, allow_input_downcast=True)
test_prediction = lasagne.layers.get_output(GCSm_net['output'], deterministic=True)
test_loss = lasagne.objectives.categorical_crossentropy(test_prediction,target_var)
test_loss = test_loss.mean()
test_acc = T.mean(T.eq(T.argmax(test_prediction, axis=1), target_var),dtype=theano.config.floatX)

val_fn = theano.function([input_var, target_var], [test_loss, test_acc],allow_input_downcast=True)
get_preds = theano.function([input_var], test_prediction,allow_input_downcast=True)

### Training Model

In [ ]:
n_examples = trainArrayGCSm.shape[0]
n_batches = int(np.floor(n_examples / BATCH_SIZE))

# Shuffle observations
random.seed(2020)
p = np.random.permutation(trainArrayGCSm.shape[0])
trainArrayGCSm_shuff = trainArrayGCSm[p]
trainLabelsGCSm_shuff = trainLabelsGCSm[p] - 1

for epoch in range(EPOCHS):
    for batch in range(n_batches):
        x_batch = trainArrayGCSm_shuff[batch*BATCH_SIZE: (batch+1) * BATCH_SIZE]
        y_batch = trainLabelsGCSm_shuff[batch*BATCH_SIZE: (batch+1) * BATCH_SIZE]
        train_fn(x_batch, y_batch)